In [9]:
def compute_log_likelihood(self, X_test, alpha, beta):
        mn_probs = np.zeros(X_test.shape[0])
        for k in range(beta.shape[0]):
            mn_probs_k = self._get_mixture_weight(alpha, k) * self._multinomial_prob(X_test, beta[k])
            mn_probs += mn_probs_k
        mn_probs[mn_probs == 0] = np.finfo(float).eps
        return np.log(mn_probs).sum()

$$
Computeloglikelihood(X_{test}, \alpha,\beta) = \sum_{i=1}^{|X_{test}|} \log (\sum_{k=1}^{|\beta|} GetMixtureWeight(\alpha, k). MultinomialProb(X_{test[i]}, \beta_{k}))
$$

In [10]:
def compute_bic(self, X_test, alpha, beta, log_likelihood=None):
        if log_likelihood is None:
            log_likelihood = self.compute_log_likelihood(X_test, alpha, beta)
        N = X_test.shape[0]
        return np.log(N) * (alpha.size + beta.size) - 2 * log_likelihood


$$
BIC = -2.computeloglikelihood(x_{test},\alpha, \beta) + \log(N) . (k_{\alpha}+ k_{\beta})
$$

In [ ]:
def compute_icl_bic(self, bic, gamma):
        classification_entropy = -(np.log(gamma.max(axis=1))).sum()
        return bic - classification_entropy

$$
ICL = BIC - \sum_{i=1}^{N} \log (max_{j})\gamma_{ij}
$$

In [11]:
 def _multinomial_prob(self, counts, beta, log=False):
        """
        Evaluates the multinomial probability for a given vector of counts
        counts: (N x C), matrix of counts
        beta: (C), vector of multinomial parameters for a specific cluster k
        Returns:
        p: (N), scalar values for the probabilities of observing each count vector given the beta parameters
        """
        n = counts.sum(axis=-1)
        m = multinomial(n, beta)
        if log:
            return m.logpmf(counts)
        return m.pmf(counts)

$$
log(p_{i})= \log(Multinomial(counts_{i}|n_{i},\beta))
$$

In [12]:
{    def _e_step(self, X, alpha, beta):
    """
    Performs E-step on MNMM model
    Each input is numpy array:
    X: (N x C), matrix of counts
    alpha: (K) or (NxK) in the case of individual weights, mixture component weights
    beta: (K x C), multinomial categories weights

    Returns:
    gamma: (N x K), posterior probabilities for objects clusters assignments
    """
    # Compute gamma
    N = X.shape[0]
    K = beta.shape[0]
    weighted_multi_prob = np.zeros((N, K))
    for k in range(K):
        weighted_multi_prob[:, k] = self._get_mixture_weight(alpha, k) * self._multinomial_prob(X, beta[k])

    # To avoid division by 0
    weighted_multi_prob[weighted_multi_prob == 0] = np.finfo(float).eps

    denum = weighted_multi_prob.sum(axis=1)
    gamma = weighted_multi_prob / denum.reshape(-1, 1)

    return gamma
}

\begin{align*}
\text{weightedmultiprob}_{ik} &= \text{getmixtureweight}(\alpha, k) \times \text{multinomialprob}(X, \beta_k) \quad \text{for } i = 1, 2, \ldots, N \text{ and } k = 1, 2, \ldots, K \\
weightedmultiprob[ \text{weightedmultiprob} = 0] &= \epsilon  \\
\text{denum}_i &= \sum_{k=1}^{K} \text{weightedmultiprob}_{ik} \quad \text{for } i = 1, 2, \ldots, N \\
\gamma_{ik} &= \frac{\text{weightedmultiprob}_{ik}}{\text{denum}_i} \quad \text{for } i = 1, 2, \ldots, N \text{ and } k = 1, 2, \ldots, K
\end{align*}


In [13]:
def _compute_vlb(self, X, alpha, beta, gamma):
        """
        Computes the variational lower bound
        X: (N x C), data points
        alpha: (K) or (NxK) with individual weights, mixture component weights
        beta: (K x C), multinomial categories weights
        gamma: (N x K), posterior probabilities for objects clusters assignments

        Returns value of variational lower bound
        """
        loss = 0
        for k in range(beta.shape[0]):
            weights = gamma[:, k]
            loss += np.sum(weights * (np.log(self._get_mixture_weight(alpha, k)) + self._multinomial_prob(X, beta[k], log=True)))
            loss -= np.sum(weights * np.log(weights))
        return loss

\begin{align*}
\text{loss} = &\sum_{k=1}^{K} \left[ \sum_{i=1}^{N} \gamma_{ik} \left( \log(\text{{_get_mixture_weight}}(\alpha, k)) + \text{{_multinomial_prob}}(X_i, \beta_k, \text{{log=True}}) \right) \right. \
&\left. - \sum_{i=1}^{N} \gamma_{ik} \log(\gamma_{ik}) \right]
\end{align*}